## This lab explains simple example to build AI Agent.

In [1]:
# @title Authentication

%pip install --upgrade --quiet google-cloud-aiplatform

#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.1 MB/s eta 0:00:00


In [2]:
# @title Agent initialization.

from IPython.display import display, Markdown

import vertexai
import vertexai.generative_models as generative_models
from vertexai.preview.generative_models import grounding

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

In [11]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을  참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 영화추천 : http://sec_vd.com/영화추천
  2. 음악추천 : http://sec_vd.com/음악추천
  3. 기타 : http://sec_vd.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def agent_1(chat_history, query):

  prompt =f"""
          당신은 영화 컨텐츠를 추천을 위한 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 영화 컨텐츠 추천에 대해서 영화 명 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 추천을 하는 이유를 현재 고객의 정보에 맞게 추천해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}
  """
  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],

  )

  return responses.text

def agent_2(chat_history, query):

  prompt =f"""
          당신은 음악 추천 을 위한 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 오늘 날씨에 맞는 음악을 추천해주세요.
          2. 추천하는 이유를 자세히 설명해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_3(chat_history, query):

  prompt =f"""
          아래 질문에 맞게 답해주세요. 전체 답변은 10줄을 넘지 않게 해주세요.
          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_orchestration(chat_history, query):

  classified = query_classification(chat_history, query)
  task = eval(classified)['classification']['class']

  print(f"Agent Task : {task}")

  if task == '영화추천':
    return agent_1(chat_history, query)
  elif task == '음악추천':
    return agent_2(chat_history, query)
  else:
    return agent_3(chat_history, query)


In [12]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  response = agent_orchestration(chat_history, query)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 안녕하세요.
Agent Task : 기타


AI Agent : "안녕하세요"는 한국어에서 가장 일반적인 인사말입니다. "무탈하시오?"와 같은 뜻으로 "잘 지내시나요?"와 유사합니다. 외국인이 한국어를 배울 때 처음 배우는 인사말 중 하나이며, "안녕하다"는 "평화롭다", "안전하다", "편안하다"를 의미합니다. 따라서 "안녕하세요?"는 "평화롭게 지내시나요?" 또는 "잘 지내시나요?"라고 해석할 수 있습니다. 


------------------------------------ 
사용자: 오늘 한국의 날씨에 맞는 음악 틀어주세요.
Agent Task : 음악추천


AI Agent : 오늘 한국 날씨에 딱 맞는 음악을 추천해 드릴게요! 

흐린 날씨에 딱 맞는 **잔잔하고 감성적인 음악**을 추천해 드릴게요. 

**비 오는 날 듣기 좋은 노래**들은  분위기 있는 가사와 멜로디로 흐린 날씨의 쓸쓸함을 더욱 돋보이게 해주죠. 

**에픽하이의 '비 오는 날 듣기 좋은 노래 (Feat. Colde)'**는  잔잔한 피아노 선율과 에픽하이의 감성적인 목소리가 어우러져 비 오는 날의 분위기를 한층 더욱 짙게 만들어 줍니다. 

**헤이즈의 '비도 오고 그래서 (Feat. 신용재)'**는  애절한 가사와 신용재의 호소력 짙은 목소리가 듣는 사람들의 마음을 울리는 곡으로,  비 오는 날의 감성을 더욱 깊게 만들어 줍니다. 

이 외에도 **브라운 아이즈의 '비오는 압구정', 유승우의 '비 오는 거리'**  등 많은 곡들이  비 오는 날의 감성을 자극하는 곡들이 많으니, 취향에 맞게 골라서 들어보세요. 


------------------------------------ 
사용자: 오늘이 몇일인가요 ?
Agent Task : 기타


...

------------------------------------ 
사용자: 현재 서울날씨는 어떤가요 ?
Agent Task : 기타


AI Agent : 죄송합니다. 제공된 출처에는 서울의 현재 날씨에 대한 정보가 없습니다. 서울의 현재 날씨를 알려드릴 수 없습니다. 


------------------------------------ 
사용자: what's the weather of seoul?
Agent Task : 기타


AI Agent : 서울의 날씨는 눈이 내리고 흐린 하늘입니다. 기온은 낮에는 2도, 밤에는 -2도 정도입니다. 

날씨는 끊임없이 변하기 때문에, 정확한 정보를 얻으려면 날씨 웹사이트 또는 앱을 확인하는 것이 좋습니다.


------------------------------------ 
사용자: 종료
